In [1]:
from pathlib import Path
from time import sleep 

import numpy as np

import roboticstoolbox as rtb
import spatialmath as sm

from manipylator import VisualRobot
from manipylator import utils

# Load Robot
Below we load a URDF file describing Manny the robot manipulator. A URDF is a textual representation of a robot, defining the parameters we need so we can calculate the forward & reverse kinemtics. 

In [2]:
with utils.render_robot_from_template("robots/empiric") as robot_urdf:
    manny = VisualRobot(robot_urdf)
print(manny.model)

[I 07/04/25 01:22:21.877 33] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


[Genesis] [01:22:34] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [01:22:34] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [01:22:34] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [01:22:34] [WARNING] No Intel XPU device available. Falling back to CPU for torch device.


/opt/conda/lib/python3.11/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


[Genesis] [01:22:35] [INFO] Consider setting 'performance_mode=True' in production to maximise runtime speed, if significantly increasing compilation time is not a concern.


RHI Error: Can not create Vulkan instance
[W 07/04/25 01:22:35.710 33] [misc.py:adaptive_arch_select@758] Arch=[<Arch.vulkan: 10>] is not supported, falling back to CPU


[Genesis] [01:22:37] [INFO] Running on [Intel(R) Core(TM) i7-8665U CPU @ 1.90GHz] with backend gs.cpu. Device memory: 15.25 GB.
[Genesis] [01:22:37] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [01:22:37] [WARNING] Scene.show_FPS is deprecated. Please use Scene.profiling_options.show_FPS
[Genesis] [01:22:37] [INFO] Scene <92b68e7> created.
[Genesis] [01:22:37] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <911be1f>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [01:22:37] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <25bf9c2>, morph: <gs.morphs.URDF(file='/tmp/tmpcgxjp6a8.urdf')>, material: <gs.materials.Rigid>.


[Genesis] [01:22:41] [INFO] Falling back to legacy URDF parser. Default values of physics properties may be off.
[Genesis] [01:22:41] [INFO] Applying offset to base link's pose with user provided value in morph.
[Genesis] [01:22:52] [INFO] Building scene <92b68e7>...
[Genesis] [01:23:32] [INFO] Compiling simulation kernels...
[Genesis] [01:25:32] [INFO] Building visualizer...
[Genesis] [01:25:46] [INFO] Viewer created. Resolution: 862×646, max_FPS: 60.
ERobot: measured, 6 joints (RRRRRR), dynamics, geometry, collision
┌──────┬──────────────────┬───────┬──────────────────┬─────────────────────────────────────────────────────────────────────────────┐
│ link │       link       │ joint │      parent      │                             ETS: parent to link                             │
├──────┼──────────────────┼───────┼──────────────────┼─────────────────────────────────────────────────────────────────────────────┤
│    0 │ base             │       │ BASE             │ SE3()                 

# Visualizing Manny
Using the `manny.visualizer.robot` object we can get the properties of it's links. For example, a given link's location in 3D space. Below we find the location of the end effector in when the all joint parameters are set to 0 degrees:

In [3]:
robot = manny.visualizer.robot
# robot

In [4]:
ee = robot.get_link('end_effector')
# ee

In [5]:
ee_initial_pos = ee.get_pos()
ee_initial_pos

tensor([-1.2851e-01,  1.5981e-06,  8.1101e-01])

In [6]:
ee_initial_quat = ee.get_quat()
ee_initial_quat

tensor([0.5000, 0.5000, 0.5000, 0.5000])

And we can verify that for the inverse is also true:

In [7]:
initial_qpos = robot.inverse_kinematics(
    link = ee,
    pos  = ee_initial_pos,
    quat = ee_initial_quat,
)
initial_qpos

[W 07/04/25 01:26:04.002 33] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


tensor([0., 0., 0., 0., 0., 0.])

I.e. that when setting all the joints to 0 degrees the end effector end up at `ee_initial_pos` and that if the end effector is at `ee_initial_pos` all the joints are at 0 degress.

Similarly, for an arbitrary (reachable) location in 3D space:

In [31]:
qpos = robot.inverse_kinematics(
    link = ee,
    pos  = [-4.8500e-02, -4.7547e-02,  4.0200e-01],
    quat = ee_initial_quat,
)
qpos

robot.set_dofs_position(qpos)
# print(robot.get_dofs_position())
manny.visualizer.scene.step()

# Tracing a Path in R3
Below we calculate a positions in R3 that correspond to a heart shaped curve:

In [47]:
ee_initial_pos[0].numpy()

array(-0.12851399, dtype=float32)

In [51]:
scale = 0.1
offset = 0.3
height = 0.1

shape = utils.parametric_heart_1(np.linspace(0, 2*np.pi, 100))
positions = scale * shape + offset
# Shift the heart on the x axis so it's center intersects with when arm is at zero position
positions[:,0] -= positions[0,0] - ee_initial_pos[0].numpy()
positions[:,2] = height

positions[:3]

array([[-0.12851399,  0.3       ,  0.1       ],
       [-0.12847791,  0.30060238,  0.1       ],
       [-0.12822709,  0.30238772,  0.1       ]])

Each row is the (x, y, z) coordinates of a point on the curve. This is useful if we'd like to visualize the expected path of the end effector.

In [53]:
# manny.visualizer.scene.clear_debug_objects()
manny.visualizer.scene.draw_debug_spheres(positions)



Similarly, we can calculate the joint parameters of each position:

In [54]:
robot.set_dofs_position([0.2, 0.2, 2.35, 1, 1.57, 0])

manny.visualizer.scene.step()
ee.get_pos()

tensor([-0.0357,  0.2696,  0.2888])

In [55]:
quat_pointing_down = [0, 0, 0, -1]

initial_pose = robot.inverse_kinematics(
        link = ee,
        pos  = positions[0],
        quat = quat_pointing_down,
        )

robot.set_dofs_position(initial_pose)
manny.visualizer.scene.step()

In [19]:
# Reset to zero pose
robot.set_dofs_position(initial_qpos)
manny.visualizer.scene.step()

# robot.set_dofs_position(q)
# # print(robot.get_dofs_position())
# manny.visualizer.scene.step()

In [20]:
# camera = manny.visualizer.camera

# camera.start_recording()
# # [camera.render() for i in range(10)]

# # Reset to initial curve pose
# robot.set_dofs_position(initial_pose)
# manny.visualizer.scene.step()

# camera.render()

In [28]:

qs = [robot.inverse_kinematics(
        link = ee,
        pos  = pos,
        quat = quat_pointing_down,
        ) for pos in positions]
qs[:5]

[tensor([ 0.9957, -0.6001,  2.0669,  1.0971,  1.5705, -2.1459]),
 tensor([ 0.9944, -0.6007,  2.0650,  1.0950,  1.5707, -2.1472]),
 tensor([ 0.9911, -0.6028,  2.0594,  1.0916,  1.5708, -2.1505]),
 tensor([ 0.9861, -0.6066,  2.0496,  1.0856,  1.5708, -2.1555]),
 tensor([ 0.9799, -0.6121,  2.0354,  1.0769,  1.5708, -2.1617])]

And to visualize the poses Manny would take, we can animate the path:

In [29]:
for q in qs:
    robot.set_dofs_position(q)
    # print(robot.get_dofs_position())
    manny.visualizer.scene.step()
    # camera.render()
    sleep(0.1)

# camera.stop_recording(save_to_filename='video.mp4', fps=20)

In [23]:
# Reset to zero pose
robot.set_dofs_position(initial_qpos)
manny.visualizer.scene.step()
